In [2]:
import datetime
from datetime import date, datetime
from distutils.command.config import config
import time
import mysql.connector
from mysql.connector import errorcode
from this import d
import requests
import json
import pprint
import urllib.parse

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


C:\Users\gabbe\AppData\Local\Temp\ipykernel_13884\401639078.py:3: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  from distutils.command.config import config


In [11]:
accuweatherAPIKey = "2uZn0oqJRmdwuMCWOoFiCGMYMOOWyouV"
mapboxToken = "pk.eyJ1IjoiYXJvbm5pIiwiYSI6ImNsOW9pODVxZDA5MWozcG8weDIzcHZmbDAifQ.-b2S4BKL5xZrbNeb9YPDvA"

dias_semana = ['Domingo', 'Segunda-feira', 'Terça-feira', 'Quarta-feira', 'Quinta-feira', 'Sexta-feira', 'Sábado']

def pegarCoordenadas():
    r = requests.get('http://www.geoplugin.net/json.gp')
    #verificação de localização, baseado na numeração de erro 200
    if(r.status_code != 200):
        print('Não foi possivel obter a localização.')
        return None
    else:
        try:
            #print(r.text) - dados da requisição http
            localizacao = json.loads(r.text)
            coordenadas = {}
            coordenadas['lat'] = localizacao['geoplugin_latitude']
            coordenadas['long'] = localizacao['geoplugin_longitude']
            print('coordenadas ok')
            return coordenadas
        except:
            return None 
           
            

def pegarCodigoLocal(lat,long):
    LocationAPIUrl = "http://dataservice.accuweather.com/locations/v1/cities/geoposition/search?apikey=" + accuweatherAPIKey + "&q=" + lat + "%2C" + long + "&language=pt-br"

    r = requests.get(LocationAPIUrl)
    if(r.status_code == 401):
        print('Não autorizado.')
    elif(r.status_code != 200):
        print('Não foi possível obter a localização.')
        return None
    else:
        try:
            locationResponse = json.loads(r.text)
            infoLocal = {}
            infoLocal['nomeLocal'] = locationResponse['LocalizedName'] + "," + locationResponse['AdministrativeArea']['LocalizedName'] + "." + locationResponse['Country']['LocalizedName']
            infoLocal['codigoLocal'] = locationResponse['Key']
            return infoLocal
        except:
            return None

def pegarTempoAgora(codigoLocal, nomeLocal):
    CurrentConditionsAPIUrl = "http://dataservice.accuweather.com/currentconditions/v1/" + codigoLocal + "?apikey=" + accuweatherAPIKey + "&language=pt-br"

    r = requests.get(CurrentConditionsAPIUrl)
    if(r.status_code != 200):
        print('Não foi possivel obter o clima atual')
        return None
    else:
        try:
            CurrentConditionsResponse = json.loads(r.text)
            global infoClima
            infoClima = {}
            infoClima['textoClima'] = CurrentConditionsResponse[0]['WeatherText']
            infoClima['temperatura'] = CurrentConditionsResponse[0]['Temperature']['Metric']['Value'] 
            infoClima['nomeLocal'] = nomeLocal
            return infoClima
        except:
            return None

def pegarPrevisao5Dias(codigoLocal):
    DailyAPIUrl = " http://dataservice.accuweather.com/forecasts/v1/daily/5day/" + codigoLocal + "?apikey=" + accuweatherAPIKey + "&language=pt-br&metric=true"

    r = requests.get(DailyAPIUrl)
    if(r.status_code != 200):
        print('Não foi possivel obter o clima atual.')
        return None
    else:
        try:
            DailyResponse = json.loads(r.text)
            infoClima5Dias = []
            for dia in DailyResponse['DailyForecasts']:
                climaDia = {}
                climaDia['max']=dia['Temperature']['Maximum']['Value']
                climaDia['min']=dia['Temperature']['Maximum']['Value']
                climaDia['clima']=dia['Day']['IconPhrase']
                diaSemana = int(date.fromtimestamp(dia['EpochDate']).strftime("%w"))
                climaDia['dia']=dias_semana[diaSemana]
                infoClima5Dias.append(climaDia)
            return infoClima5Dias
        except:
            return None


In [12]:
coordenadas = pegarCoordenadas()
local = pegarCodigoLocal(coordenadas.get('lat'),coordenadas.get('long'))
print(local)
climaAtual = pegarTempoAgora(local['nomeLocal'])

sql = ('INSERT INTO localGps(latitude,longitude) VALUES (%s,%s)')
values = (coordenadas.get('lat'), coordenadas.get('long'))


coordenadas ok
Não foi possível obter a localização.
None


TypeError: 'NoneType' object is not subscriptable

In [ ]:
try:
    coordenadas = pegarCoordenadas()
    print(coordenadas)
    localiza = (coordenadas['lat'],coordenadas['long'])
    lat = (coordenadas['lat'])
    long = (coordenadas['long'])
    print(pegarCodigoLocal)
    print(localiza)
except:
        print('Não foi possível obter a previsão para este local.')

try:
    local = pegarCodigoLocal(coordenadas['lat'],coordenadas['long'])
    climaAtual = pegarTempoAgora(local['codigoLocal'], local['nomeLocal'])
    print(infoClima['temperatura'])
    print(infoClima['textoClima'])

except:
     print('Erro ao obter o clima atual.')


: 